# For the purpose of learning how to build, I decided to start with the simplest method of build chatbots (i think)

In [14]:
import en_core_web_lg # Large SpaCy model for English language
import numpy as np # numpy
import re # Regular expression
import spacy # NLU - Natural Language understanding.
from collections import defaultdict
from sklearn.svm import SVC # support vector classification model

In [15]:
output_format = "IN:  {input}\nOUT: {output}\n" + "_"*50

In [16]:
# coded exact questions 
responses_exact = {
    "what would you like to eat tonight?": "Pasta with salmon and red pesto please!",
    "what time will you be home tonight?": "I will be home around 6 pm.",
    "default": "I love you too!"
}

def respond_exact(text):
    response = responses_exact.get(text.lower(), responses_exact['default'])
    return(output_format.format(input=text, output=response))

In [17]:
print(respond_exact("What would you like to eat tonight?"))
print("_"*50)
print(respond_exact("What time will you be home tonight?"))
print("_"*50)
print(respond_exact("I just found out my ex ran away."))

IN:  What would you like to eat tonight?
OUT: Pasta with salmon and red pesto please!
__________________________________________________
__________________________________________________
IN:  What time will you be home tonight?
OUT: I will be home around 6 pm.
__________________________________________________
__________________________________________________
IN:  I just found out my ex ran away.
OUT: I love you too!
__________________________________________________


## Version 2: Pattern Matching

In [18]:
# Define keywords that can help determine the intent 
intent_keywords = {
    'dinner_preference': ['eat', 'dinner', 'food', 'cook', 'craving'],
    'arrival_time': ['time', 'when', 'get here', 'be home']
}
# Create a dictionary of patterns
patterns = {intent: re.compile('|'.join(keys)) for intent, keys in intent_keywords.items()}

# Define a function to find the intent of a message
def get_intent_re(message):
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            return(intent)
    else:
        return('default')

responses_re = {
    "dinner_preference":"Pasta with salmon and red pesto please!",
    "arrival_time": "I will be home around 6 pm.",
    "default":"I like you too!"
}

def respond_re(text):
    response = responses_re.get(get_intent_re(text))
    return(output_format.format(input=text, output=response))

In [19]:
print(respond_re("what would you like to eat tonight?"))
print(respond_re("what time will you be home tonight?"))
print(respond_re("I just food out my boss is leaving the company."))


IN:  what would you like to eat tonight?
OUT: Pasta with salmon and red pesto please!
__________________________________________________
IN:  what time will you be home tonight?
OUT: I will be home around 6 pm.
__________________________________________________
IN:  I just food out my boss is leaving the company.
OUT: Pasta with salmon and red pesto please!
__________________________________________________


### A bit of machine learning :D

In [20]:
# Create training data
training_sentences = [
    "What would you like to have for dinner?",
    "What do you want to eat tonight?",
    "I don't know what to cook tonight.",
    "Do you have any cravings?",
    "Can I get you something to eat?", 
    "What time will you be home?",
    "How much longer will you be?",
    "When can we expect you to get here?",
    "What's taking you so long?",
    "At what hour will you be here?"
    
]
training_intents = [
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "dinner_preference",
    "arrival_time",
    "arrival_time",
    "arrival_time",
    "arrival_time",
    "arrival_time"   
]

In [22]:
# this may take a couple of seconds
nlp = en_core_web_lg.load()

In [23]:
# Initialize the array with zeros: X
X_train = np.zeros((len(training_sentences), 
              nlp('sentences').vocab.vectors_length))

for i, sentence in enumerate(training_sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    # Save the document's .vector attribute to the corresponding row in X
    X_train[i, :] = doc.vector

In [24]:
# Create a support vector classifier
clf = SVC(C=1, gamma="auto", probability=True)

# Fit the classifier using the training data
clf.fit(X_train, training_intents)

#Yes, a lot can be done here to check / improve model performance! We will leave that for another day!

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [25]:
def get_intent_ml(text):
    doc = nlp(text)
    return(clf.predict([doc.vector])[0])


In [26]:
responses_ml = {
    "dinner_preference":"Pasta with salmon and red pesto please!",
    "arrival_time": "I will be home around 6 pm.",
    "default":"I like you too!"
}

def respond_ml(text):
    response = responses_ml.get(get_intent_ml(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))


In [27]:
print(respond_ml("what would you like to eat tonight?"))
print(respond_ml("what time will you be home tonight?"))
print(respond_ml("l"))

IN:  what would you like to eat tonight?
OUT: Pasta with salmon and red pesto please!
__________________________________________________
IN:  what time will you be home tonight?
OUT: I will be home around 6 pm.
__________________________________________________
IN:  l
OUT: Pasta with salmon and red pesto please!
__________________________________________________


In [28]:
# add this to responses dict: "default": "I love you too!"
# in the predict function: if the model is not too sure about the intent, return the string "default"
    # There is a function that gives the probabilities for each of the possible outputs
    # If the maximum probability is low, one might say that the model is not sure about the intent
    # Note! This idea should work, but relies on the functionality of the predict_proba function:
    # for the SVC model, the predict_proba function does not give meaningfull results for small datasets:
    # http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict_proba

def get_intent_ml_2(text):
    """
        Returns the intent from a given text, unless the model is not sure, in which case 'default' is returned
    """
    doc = nlp(text)
    max_proba = max(clf.predict_proba([doc.vector])[0])
    if(max_proba == 0.5):
        return('default')
    else:
        return(clf.predict([doc.vector])[0])

def respond_ml_2(text):
    response = responses_ml.get(get_intent_ml_2(text), responses_ml["default"])
    return(output_format.format(input=text, output=response))

######################################################
print(respond_ml(  'flowers'))
print(respond_ml_2('flowers'))

IN:  flowers
OUT: Pasta with salmon and red pesto please!
__________________________________________________
IN:  flowers
OUT: I like you too!
__________________________________________________


In [29]:
def get_all_entities(text):
    """
        Get all entities in a given text, in a text: label_ dictionary
    """
    doc = nlp(text)
    
    d = defaultdict(list)
    for ent in doc.ents:
        d[ent.label_].append(ent.text)
    return(d)

In [30]:
test_ents = get_all_entities('what would you like to eat tonight?, or next tuesday or wednesday fish football Bengals')
print(sorted(test_ents.items()))


[('DATE', ['next tuesday', 'wednesday']), ('GPE', ['Bengals']), ('TIME', ['tonight'])]


In [31]:
policy = {
    ("dinner_preference", "time and date"): "I want to eat pasta",
    ("dinner_preference", "time only"): "I want to eat pasta",
    ("dinner_preference", "date only"): "I want to eat pasta",
    ("dinner_preference", "none"): "When?",
    ("arrival_time", "time and date"): "I will be home at six",
    ("arrival_time", "time only"): "I will be home at six",
    ("arrival_time", "date only"): "I will be home at six",
    ("arrival_time", "none"): "When?",
    ("default", "none"): "What do you want?",
}

In [32]:
def respond_ml_3(text):
    """Check for specification of date and time
        If not specified, ask for clarification
    """
    intent = get_intent_ml_2(text)
        
    if intent != 'default':
        entities = get_all_entities(text)
        if 'TIME' in entities and 'DATE' in entities:
            specification = 'time and date'
            time = ' and '.join(entities['DATE']) + ' at ' + ' and '.join(entities['TIME'])
        elif 'TIME' in entities:
            specification = 'time only'
            time = ' and '.join(entities['TIME'])
        elif 'DATE' in entities:
            specification = 'date only'
            time = ' and '.join(entities['DATE'])
        else:
            specification = 'none'
            time = ""
    else:
        specification = 'none'
        time = ""
    
    response = policy.get((intent, specification)) + ' ' + time
    return(output_format.format(input=text, output=response))

In [36]:
print(respond_ml_3('what would you like to eat next wednesday thursday and friday?'))

IN:  what would you like to eat next wednesday thursday and friday?
OUT: I want to eat pasta next wednesday and thursday and friday
__________________________________________________


### The end